# 7 operate turtlebot3 using gamepad and web controller

## operate turtlebot3 using gamepad

#### prepare raspberry pi connected to gamepad

1. ssh to raspberry pi connected to gamepad.
1. clone repositories of gamepad controller from github.
    * https://github.com/tech-sketch/fiware-gamepad-controller.git

#### setup cert file to connect MQTT broker of fiware

1. copy the `ca.crt` from local to raspberry pi

```bash
$ scp ${PJ_ROOT}/secrets/ca.crt ${user}@${raspberry_pi}:${repogitory_root}/secrets/ca.crt
```

#### setup the parameters of MQTT broker

1. start a terminal on Ubuntu desktop.
1. setup `mqtt.yaml`

```bash
raspberrypi:$ ls conf/pxkwcr-azure.yaml.template conf/pxkwcr-azure.yaml
raspberrypi:$ vi conf/pxkwcr-azure.yaml
```

#### install python libraries for bridge node

1. install python libraries for bridge node

```bash
raspoberypi:$ pip install -r requirements/common.txt
```

#### run main program

1. run program

```bash
raspberrypi$ ./main.py pxkwcr-azure
```

* confirm that the raspberry pi connected MQTT broker successfully.

```
2018/07/19 14:20:12 [   INFO] src.controller - connected mqtt broker[mqtt.cloudconductor.jp:8883], response_code=0
```

if `response_code` is not 0, `pxkwcr-azure.yaml` and/or `ca.crt` is invalid.

#### control turtlebot3

1. press any button of gamepad.
1. turtlebot3 should move according the pressed button.

## operate turtlebot3 using web controller

#### open web controller
change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/container-centric-fiware-demonstration"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/container-centric-fiware-demonstration
```

In [ ]:
source ${PJ_ROOT}/docs/azure_aks/env

In [ ]:
open https://api.${DOMAIN}/controller/web/

#### confirm basic auth username & password

In [ ]:
cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.basic_auths | map(select(.allowed_paths[] | contains ("/controller/web"))) | .[0].username' -r

In [ ]:
cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.basic_auths | map(select(.allowed_paths[] | contains ("/controller/web"))) | .[0].password' -r

#### control turtlebot3

1. login the opened page using above user & password.
1. press any button of web controller.
1. turtlebot3 should move according the pressed button.